<a href="https://colab.research.google.com/github/gzmlab/github-essentials/blob/master/Library_Analyse_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Original source: https://towardsdatascience.com/exploratory-data-analysis-with-pandas-and-jupyter-notebooks-36008090d813

In [0]:
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')
df = pd.read_csv('/content/gdrive/My Drive/00-Data Scientist and ML Engineer/TWDS-Exploratory Data Analysis with Pandas using Jupyter Notebooks/random_people.csv')

nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
There are 350 rows and 4 columns


In [0]:
#start getting a feel of the data
df.head(10)

,Unnamed: 0,name,surname,salary
0,0,John,Green,6500
1,1,Albert,Joneson,14500
2,2,Richard,Black,12000
3,3,William,Green,8000
4,4,Richard,Green,8000
5,5,Henry,Joneson,8000
6,6,John,Black,12500
7,7,Richard,White,8000
8,8,William,Green,14500
9,9,Richard,Black,10000


In [0]:
df['salary'].value_counts()

14500    74
8000     65
6500     45
10500    35
6000     34
12500    34
10000    33
12000    30
Name: salary, dtype: int64

In [0]:
df['salary'].median()

10000.0

In [0]:
df["salary_after_tax"] = df["salary"]*.8

In [0]:
def tax(s):
    if s>=6000:
        return s*.7
    else:
        return s*.85

In [0]:
df["salary_after_tax"] = df["salary"].apply(tax)

In [0]:
df.head(10)

,Unnamed: 0,name,surname,salary,salary_after_tax
0,0,John,Green,6500,4550.0
1,1,Albert,Joneson,14500,10150.0
2,2,Richard,Black,12000,8400.0
3,3,William,Green,8000,5600.0
4,4,Richard,Green,8000,5600.0
5,5,Henry,Joneson,8000,5600.0
6,6,John,Black,12500,8750.0
7,7,Richard,White,8000,5600.0
8,8,William,Green,14500,10150.0
9,9,Richard,Black,10000,7000.0


In [0]:
df_low = df[df["salary"]<=6500]
df_high = df[df["salary"]>6500]

In [0]:
df_low.head(10)

,Unnamed: 0,name,surname,salary,salary_after_tax
0,0,John,Green,6500,4550.0
10,10,Albert,Green,6500,4550.0
23,23,Henry,White,6500,4550.0
29,29,Richard,Goodman,6000,4200.0
39,39,Albert,Black,6500,4550.0
45,45,Henry,Joneson,6500,4550.0
47,47,Richard,Green,6500,4550.0
48,48,Richard,Black,6500,4550.0
52,52,Henry,White,6500,4550.0
58,58,Henry,Black,6500,4550.0


In [0]:
df_high["salary"].mean()

11348.70848708487

In [0]:
df_low= df.loc[df["salary"]<=6500,"salary"]
df.loc[df["salary"]<=6500,"salary_after_tax"] = df_low*.85

df_low= df.loc[df["salary"]>6500,"salary"]
df.loc[df["salary"]>6500,"salary_after_tax"] = df_low*.7

In [0]:
df_low

1      14500
2      12000
3       8000
4       8000
5       8000
       ...  
342     8000
343    14500
344    12000
346     8000
348    12000
Name: salary, Length: 271, dtype: int64

In [0]:
df.head(10)

,name,surname,salary
0,John,Black,9000
1,John,Green,10500
2,Henry,Black,12000
3,John,Black,9000
4,William,White,10500
5,Albert,White,5500
6,John,White,11500
7,John,White,12500
8,Albert,White,11500
9,Henry,Green,12500


In [0]:
# script to generate random file
import pandas as pd
import random

names = ["Albert","John","Richard","Henry","William"]
surnames = ["Goodman","Black","White","Green","Joneson"]
salaries = [500*random.randint(10,30) for _ in range(10)]

def generate_random_person(names, surnames, salaries):
    return {"name":random.sample(names,1)[0],
            "surname":random.sample(surnames,1)[0],
            "salary":random.sample(salaries,1)[0]}
def generate_people(k):
    return [generate_random_person(names, surnames, salaries) for _ in range(k)]

df = pd.DataFrame(generate_people(350),columns=["name","surname","salary"])
df
#df.to_csv("random_people.csv")

,name,surname,salary
0,John,Black,9000
1,John,Green,10500
2,Henry,Black,12000
3,John,Black,9000
4,William,White,10500
...,...,...,...
345,William,Joneson,12500
346,William,Joneson,12500
347,Richard,Joneson,9000
348,Richard,Green,12500


In [0]:
# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Create & upload a file text file.
# uploaded = drive.CreateFile({'title': 'Sample file.txt'})

tgt_folder_id = '1kOM0PQfxKVPCH21oVLukCUwfzTFclb-0'
# it corresponds to the path '/content/gdrive/My Drive/00-Data Scientist and ML Engineer/TWDS-Exploratory Data Analysis with Pandas using Jupyter Notebooks'

uploaded = drive.CreateFile({
        'title': 'df content.csv',
        "mimeType": "text/csv",
        'parents': [{'id': tgt_folder_id}] }) 

import io
s = io.StringIO()
df.to_csv(s)
s.getvalue()
uploaded.SetContentString(s.getvalue())
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# 3. Load a file by ID and print its contents.
downloaded = drive.CreateFile({'id': uploaded.get('id')})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Uploaded file with ID 1Th868mFwciz6hzAaA5SHcVAiDMQnMnQO


In [0]:
!apt-get -qq install -y spacy

E: Unable to locate package spacy


In [0]:
%%html
<a href="http://example.com">link</a>

#Interactive Widgets
[Pluralsight](https://app.pluralsight.com/course-player?clipId=d59a222d-c7eb-49cd-a550-49cb63248ee6)

In [0]:
!pip install ipywidgets

In [0]:
!jupyter nbextension enable --py -widgetsnbextension --sys-prefix

In [0]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [12]:
w = widgets.IntSlider(value=10,
                      min=-5,
                      max=20,
                      step=1,
                      description='Range Slider:',
                      continuous_update=False,
                      orientation='horizontal')
w

IntSlider(value=10, continuous_update=False, description='Range Slider:', max=20, min=-5)

In [14]:
w.value

6

In [16]:
r = widgets.IntRangeSlider(value=[10,1000],
                      min=0,
                      max=10000,
                      step=1,
                      description='Price Slider:',
                      orientation='horizontal')
r

IntRangeSlider(value=(10, 1000), description='Price Slider:', max=10000)

In [18]:
r.value

(3509, 8272)

In [24]:
p = widgets.IntProgress(value=70,
                      min=0,
                      max=100,
                      step=1,
                      description='Loading:',
                      bar_style='success',
                      orientation='horizontal')
p

IntProgress(value=70, bar_style='success', description='Loading:')

In [0]:
import time
for i in range(0,110,10):
  p.value =i
  time.sleep(1)

In [32]:
t = widgets.BoundedIntText(value=5,
                      min=0,
                      max=100,
                      step=1,
                      description='Test:',
                      disabled=False)
t

BoundedIntText(value=5, description='Test:')

In [33]:
t.value

6

In [35]:
widgets.Checkbox(value=False, description='Check Me')

Checkbox(value=False, description='Check Me')

In [0]:
 dd = widgets.Dropdown(
    options=['None', '1', '2', '3'],
    value='None',
    description='Number:',
    disabled=False)

In [42]:
dd

Dropdown(description='Number:', index=1, options=('None', '1', '2', '3'), value='1')

In [43]:
dd.value

'1'

In [48]:
rb = widgets.RadioButtons(
    options=['pepperoni', 'pineapple', 'anchovies', 'spaghetti', 'macaroni'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Pizza topping:',
    disabled=False
)
rb

RadioButtons(description='Pizza topping:', options=('pepperoni', 'pineapple', 'anchovies', 'spaghetti', 'macar…

In [49]:
rb.value

'macaroni'

In [0]:
button = widgets.Button(
    description='Happiness button',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

In [0]:
def button_click_event_handler(button):
  print('Youclicked! Goog things are about to happen!')
  print(button.description)

In [0]:
button.on_click(button_click_event_handler)

In [0]:
button

#Addint interactivity to Notebook
[Pluralsight](https://app.pluralsight.com/course-player?clipId=8b9004be-24d3-4081-8d0d-7fe73722ff14)

In [65]:
def f(x):
  return x
interact(f, x=10);

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

In [66]:
interact(f, x=True);

interactive(children=(Checkbox(value=True, description='x'), Output()), _dom_classes=('widget-interact',))

In [67]:
interact(f, x='Hello Widgets');

interactive(children=(Text(value='Hello Widgets', description='x'), Output()), _dom_classes=('widget-interact'…

In [68]:
@interact(x=True, y=5.0)
def g(x,y):
  return (x, y)

interactive(children=(Checkbox(value=True, description='x'), FloatSlider(value=5.0, description='y', max=15.0,…

In [0]:
def h(p, q):
  return (p, q)

In [70]:
interact(h, p=5, q=fixed(20))

interactive(children=(IntSlider(value=5, description='p', max=15, min=-5), Output()), _dom_classes=('widget-in…

<function __main__.h>

In [0]:
import matplotlib.pyplot as plt
import numpy as np

In [0]:
def f(m, b):
  plt.figure(2)
  x = np.linspace(-10, 10, num=1000)
  plt.plot(x, m *x + b)
  plt.ylim(-5, 5)
  plt.show()

interactive_plot = interactive(f, m = (-2.0,2.0), b=(-3, 3, 0.5))

In [0]:
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot


In [77]:
from IPython.display import display
text = widgets.Text()
display(text)
def make_upper_case(input_text):
  text.value = input_text.value.upper()
  print(text.value )

text.on_submit(make_upper_case)

Text(value='')

D<DCDSC


In [78]:
import pandas as pd

from google.colab import drive
import pandas as pd

drive.mount('/content/gdrive')
nba = pd.read_csv('/content/gdrive/My Drive/00-Data Scientist and ML Engineer/02-Data Analysis with Pandas and Python/original/nba.csv')

nRow, nCol = nba.shape
print(f'There are {nRow} rows and {nCol} columns')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
There are 458 rows and 9 columns


In [79]:
nba.sample(10)

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
423,Andre Roberson,Oklahoma City Thunder,21.0,SG,24.0,6-7,210.0,Colorado,1210800.0
88,Marreese Speights,Golden State Warriors,5.0,C,28.0,6-10,255.0,Florida,3815000.0
287,Alonzo Gee,New Orleans Pelicans,15.0,SF,29.0,6-6,225.0,Alabama,1320000.0
49,Joel Embiid,Philadelphia 76ers,21.0,C,22.0,7-0,250.0,Kansas,4626960.0
104,Austin Rivers,Los Angeles Clippers,25.0,PG,23.0,6-4,200.0,Duke,3110796.0
430,Allen Crabbe,Portland Trail Blazers,23.0,SG,24.0,6-6,210.0,California,947276.0
248,Andrew Goudelock,Houston Rockets,0.0,PG,27.0,6-3,200.0,Charleston,200600.0
183,Steve Blake,Detroit Pistons,22.0,PG,36.0,6-3,172.0,Maryland,2170465.0
139,Caron Butler,Sacramento Kings,31.0,SF,36.0,6-7,228.0,Connecticut,1449187.0
43,Sasha Vujacic,New York Knicks,18.0,SG,32.0,6-7,195.0,NaN,947276.0


In [81]:
@interact
def show_age_more_than(column='Age', x = 25):
  return nba.loc[nba[column] > x]

interactive(children=(Text(value='Age', description='column'), IntSlider(value=25, description='x', max=75, mi…